In [1]:
import pandas as pd
import shutil 
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

import sys
sys.path.append('../../preprocess/single_slice')
from split import box2diameter
sys.path.append('../')
from jsonio import load

In [28]:
mindiam = 8
source_base = '/data/liumingzhou/CounterAlign_output/preprocess/multiple_slices/mindiam7_fulllung'
target_base = '/data/liumingzhou/CounterAlign_output/preprocess/multiple_slices/mindiam{}_fulllung_cls'.format(mindiam)
os.makedirs(target_base,exist_ok=True)

In [29]:
source_annos = pd.read_csv(os.path.join(source_base,'annos.csv')).set_index('filename')
raw_annos = pd.read_csv(os.path.join(source_base.replace(source_base.split('/')[-1],''),'raw.csv')).set_index('filename')
target_annos = deepcopy(source_annos[source_annos['box'].map(box2diameter)>=mindiam])

#### add attributes to target_annos

#### check attribution balancing
attributes annotations are generated in http://162.105.162.24:8889/notebooks/Projects/Counter_align/lidc/hierarchical_model/1_extract_attributes.ipynb
not here any more

#### pick those index with (1) jitid==0  (2) middle 1/3 slices

In [30]:
record = dict()
for index in target_annos.index:
    _,_,pid,nid,sliceid = index.split('-')
    
    key = pid+'-'+nid
    if key not in record.keys():
        record[key] = list()
    record[key].append(int(sliceid))

In [31]:
indices = list()
for key in record.keys():
    record[key].sort()
    # pick the middle 1/3 slices
    length = len(record[key])
    for sliceid in record[key][length//2-length//6:length//2+length//6+1]:
        index = 'LIDC-IDRI-{}-{}'.format(key,sliceid)
        indices.append(index)

In [32]:
target_annos = target_annos.loc[indices]
target_annos.to_csv(os.path.join(target_base,'annos.csv'))
print('Ratio of malignant nodules: {:.3f}'.format(sum(target_annos['malignancy'].tolist())/len(target_annos['malignancy'].tolist())))

Ratio of malignant nodules: 0.648

In [33]:
for subset in ['train','val','test']:
    os.makedirs(os.path.join(target_base,subset),exist_ok=True)
    src_filenames = os.listdir(os.path.join(source_base,subset))
    for file in tqdm(src_filenames,desc=subset):
        nid = file.split('.')[0]
        if nid in target_annos.index:
            src = os.path.join(source_base,subset,file)
            tag = os.path.join(target_base,subset,file)
            shutil.copy(src,tag)

test: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3215/3215 [00:00<00:00, 17292.72it/s]


In [34]:
print('number train: {}, val: {}'.format(len(os.listdir(os.path.join(target_base,'train'))),
                                       len(os.listdir(os.path.join(target_base,'val')))))

number train: 2008, val: 653